In [ ]:
USE DATABASE cortex_analyst_demo;
USE SCHEMA cortex_analyst_demo.revenue_timeseries;

-- Fact table: daily_revenue
CREATE OR REPLACE TABLE cortex_analyst_demo.revenue_timeseries.daily_revenue (
    date DATE,
    revenue FLOAT,
    cogs FLOAT,
    forecasted_revenue FLOAT,
    product_id INT,
    region_id INT
);

In [ ]:
USE DATABASE cortex_analyst_demo;
USE SCHEMA cortex_analyst_demo.revenue_timeseries;

-- Dimension table: product_dim
CREATE OR REPLACE TABLE cortex_analyst_demo.revenue_timeseries.product_dim (
    product_id INT,
    product_line VARCHAR(16777216)
);

In [ ]:
USE DATABASE cortex_analyst_demo;
USE SCHEMA cortex_analyst_demo.revenue_timeseries;

-- Dimension table: region_dim
CREATE OR REPLACE TABLE cortex_analyst_demo.revenue_timeseries.region_dim (
    region_id INT,
    sales_region VARCHAR(16777216),
    state VARCHAR(16777216)
);

In [ ]:
import modin.pandas as pd
import streamlit as st
# Import the Snowpark plugin for modin.
import snowflake.snowpark.modin.plugin
# Import Snowpark session
from snowflake.snowpark.context import get_active_session


In [ ]:
# Retrieve the Session
session = get_active_session()

# Set context
session.use_schema('cortex_analyst_demo.revenue_timeseries')

In [ ]:
df_d_rev = pd.read_csv('@RAW_DATA/daily_revenue.csv')
df_prod = pd.read_csv('@RAW_DATA/product.csv')
df_reg = pd.read_csv('@RAW_DATA/region.csv')

In [ ]:
df_reg_1 = df_d_rev[df_d_rev['Region_id'] == 1]

df_reg_1['REVENUE'].sum()


In [ ]:
st.bar_chart(df_d_rev.groupby('Region_id')['REVENUE'].sum().to_pandas())

In [ ]:
df_total_p_line_rev = df_d_rev.join(df_prod.set_index('Product_id'), 
                                    on='Product_id').groupby('Product_line')['REVENUE'].sum()
st.bar_chart(df_total_p_line_rev.to_pandas())

In [ ]:
# Save data to region_dim table
df_reg.to_snowflake(
    name = 'cortex_analyst_demo.revenue_timeseries.region_dim',
    if_exists='replace',
    index=False
)

# Save data to product_dim table
df_prod.to_snowflake(
    name = 'cortex_analyst_demo.revenue_timeseries.product_dim',
    if_exists='replace',
    index=False
)

df_d_rev.to_snowflake(
    name = 'cortex_analyst_demo.revenue_timeseries.daily_revenue',
    if_exists='replace',
    index=False
)


In [ ]:
select * from cortex_analyst_demo.revenue_timeseries.daily_revenue;